In [9]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from google import genai
import json
from pydantic import BaseModel
import time
import pymupdf
import tempfile
from tqdm import tqdm
import sqlite3

In [43]:
anexos = pd.read_excel('docs/jusbrasil.xlsx', sheet_name="Anexos")

In [44]:
anexos_copia = anexos.copy().loc[:,["processoID", "processoAnexoID", "Download copia", "Tipo de anexo", "Publicado em"]]
sentencas = anexos_copia.loc[anexos["Tipo de anexo"].isin(["SENTENCA"]),
                                           ["processoID", "processoAnexoID", "Download copia", "Publicado em"]]
sentencas.head()

,processoID,processoAnexoID,Download copia,Publicado em
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,2021-08-13
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,2022-05-16
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...,2022-07-20
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...,2024-07-31
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...,2025-01-29


In [45]:
def identifica_acesso_negado(processos):
    copia_processos = processos.copy()
    acessos_negados = []

    for i in range(copia_processos.shape[0]):
        link = copia_processos.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""

        if 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'Acesso negado', texto)
        if match:
            acessos_negados.append(i)
            continue
    
    return copia_processos.iloc[acessos_negados, :]

df_html_only = sentencas[sentencas['Download copia'].str.contains(r'\.html?$', case=False, na=False)]
acessos_negados = identifica_acesso_negado(df_html_only)
acessos_negados.head()

,processoID,processoAnexoID,Download copia,Publicado em
1293,471515190,727395468,http://jud-anexos.digesto.com.br/3a9fbbe2c4f69...,2022-04-28
1302,471515190,727395467,http://jud-anexos.digesto.com.br/7767cbbc02e35...,2024-03-07
2876,595998611,1403409515,http://jud-anexos.digesto.com.br/0ec3e0c015d72...,2024-08-05
2877,595998611,1403409516,http://jud-anexos.digesto.com.br/385306277d658...,2024-08-05
8000,690025576,2015456650,http://jud-anexos.digesto.com.br/fb01c9fd839aa...,2021-01-01


In [46]:
# Dropando os acessos negados
# sentencas_acessaveis = sentencas[sentencas["processoAnexoID"].isin(acessos_negados["processoAnexoID"]) == False]

# Exportando sentenças sem acesso negado para um arquivo Excel
# sentencas_acessaveis.to_excel("docs/sentencas_acessaveis.xlsx", index=False)
sentencas_acessaveis = pd.read_excel("docs/sentencas_acessaveis.xlsx")

In [47]:
# Verificando quantidade de processos após o drop
sentencas_acessaveis["processoID"].unique().shape

(223,)

In [51]:
ids_processos = sentencas_acessaveis["processoID"].unique()
processos_com_mais_de_uma_sentenca = []

for id in ids_processos:
    linhas_correspondentes = sentencas_acessaveis.loc[sentencas_acessaveis['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        processos_com_mais_de_uma_sentenca.append(int(id))

print(f"Número de processos com mais de uma sentença: {len(processos_com_mais_de_uma_sentenca)}")

Número de processos com mais de uma sentença: 84


In [52]:
sentencas_final = sentencas_acessaveis.copy()
for id in ids_processos:
    linhas_correspondentes = sentencas_acessaveis.loc[sentencas_acessaveis['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        if linhas_correspondentes['Publicado em'].nunique() == 1:
            # Data de publicação igual: eliminando por ordem de anexo id
            linhas_correspondentes = linhas_correspondentes.sort_values(by=["processoAnexoID"], ascending=True)
            manter_id = linhas_correspondentes.iloc[0]['processoAnexoID']
        else:
            # Ordenando as linhas por data
            linhas_correspondentes = linhas_correspondentes.sort_values(by=["Publicado em"], ascending=True)
            # Mantendo a primeira linha (mais antiga)
            manter_id = linhas_correspondentes.iloc[0]['processoAnexoID']
        
        # Eliminar todas as outras com mesmo processoID e processoAnexoID diferente do que foi mantido
        sentencas_final = sentencas_final[~((sentencas_final["processoID"] == id) & (sentencas_final["processoAnexoID"] != manter_id))]

In [53]:
# Verificando dataframe após a eliminação
ids_processos = sentencas_final["processoID"].unique()
processos_com_mais_de_uma_sentenca = []

for id in ids_processos:
    linhas_correspondentes = sentencas_final.loc[sentencas_final['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        processos_com_mais_de_uma_sentenca.append(int(id))

print(f"Número de processos com mais de uma sentença: {len(processos_com_mais_de_uma_sentenca)}")

Número de processos com mais de uma sentença: 0


In [54]:
# Verificando se a quantidade de processos após o drop se manteve
sentencas_final["processoID"].unique().shape

(223,)

In [55]:
# Exportando df final de sentenças para um arquivo Excel
# sentencas_final.to_excel("docs/sentencas_final.xlsx", index=False)
sentencas_final = pd.read_excel("docs/sentencas_final.xlsx")

In [56]:
def verifica_dano_ambiental(texto):
    """
    Função que verifica se o texto possui dano ambiental.
    Se houver, justifica a resposta. Caso contrário, responde "Não há dano ambiental".
    """
    
    prompt = f"""
    Você é um especialista em direito ambiental. 
    Você deve analisar o seguinte texto e verificar se há dano ambiental.
    Se houver, justifique sua resposta em até 20 palavras. Caso contrário, responda "Não há dano ambiental".
    Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão
    de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
    Refira-se EXCLUSIVAMENTE ao texto fornecido para identificar se há dano ambiental e para justificar sua resposta!!

    Texto: {texto}
    """
    
    class FormatoResposta(BaseModel):
        isDanoAmbiental: bool
        justificativa: str

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': FormatoResposta,
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    time.sleep(3)
    return resposta

In [57]:
respostas = []

for id in tqdm.tqdm(ids_processos):
    linha = sentencas_final.loc[sentencas_final['processoID'] == id]
    texto = ''
    linha = linha.iloc[0] # extrai a linha como Series

    processoAnexoID = linha['processoAnexoID']
    link = linha['Download copia']
    response = requests.get(link)
    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    
    if 'pdf' in content_type:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto += page.get_text()
    
    elif 'html' in content_type:
        soup = BeautifulSoup(response.content, 'html.parser')
        texto += soup.get_text()

    resposta_prompt = verifica_dano_ambiental(texto)
    resposta_prompt = json.loads(resposta_prompt.text)
    resposta_prompt["processoAnexoID"] = processoAnexoID
    resposta_prompt["link_referencia"] = link
    resposta_prompt["processoID"] = id
    respostas.append(resposta_prompt)

respostas

[{'isDanoAmbiental': True,
  'justificativa': 'A empresa mantinha madeira ilegal em depósito, o que causa dano ambiental por fomentar a extração ilegal.',
  'processoAnexoID': np.int64(2344968081),
  'link_referencia': 'http://jud-anexos.digesto.com.br/0cacd6d80c499ae25dcb85380a07c3dd.pdf',
  'processoID': np.int64(474474022)},
 {'isDanoAmbiental': True,
  'justificativa': 'O texto refere-se à destruição de 121,15 hectares de floresta nativa, configurando dano ao meio ambiente.',
  'processoAnexoID': np.int64(2294638461),
  'link_referencia': 'http://jud-anexos.digesto.com.br/90df1f9ac9917f6df4b9f91915b3a8bd.pdf',
  'processoID': np.int64(474474806)},
 {'isDanoAmbiental': False,
  'justificativa': 'Não há dano ambiental. O texto trata de uma ação judicial sobre a venda de um aparelho sem carregador.',
  'processoAnexoID': np.int64(1511375012),
  'link_referencia': 'http://jud-anexos.digesto.com.br/c89c7a26febccd44703bb340de170f09.pdf',
  'processoID': np.int64(575718248)},
 {'isDanoAmb

In [59]:
# Transformando a lista de respostas em um DataFrame
respostas_df = pd.DataFrame(respostas)

# Reorganizando as colunas
respostas_df = respostas_df[["processoID", "processoAnexoID", "isDanoAmbiental", "justificativa", "link_referencia"]]

respostas_df.head()

,processoID,processoAnexoID,isDanoAmbiental,justificativa,link_referencia
0,474474022,2344968081,True,"A empresa mantinha madeira ilegal em depósito,...",http://jud-anexos.digesto.com.br/0cacd6d80c499...
1,474474806,2294638461,True,"O texto refere-se à destruição de 121,15 hecta...",http://jud-anexos.digesto.com.br/90df1f9ac9917...
2,575718248,1511375012,False,Não há dano ambiental. O texto trata de uma aç...,http://jud-anexos.digesto.com.br/c89c7a26febcc...
3,580649179,1504080402,False,"Não há dano ambiental, pois o texto trata de i...",http://jud-anexos.digesto.com.br/585fdfebfdd6d...
4,654928926,2419043700,False,Não há dano ambiental. O texto trata de cobran...,http://jud-anexos.digesto.com.br/6be7e9beac86d...


In [2]:
# Exportando df de classificação de sentenças para um arquivo Excel
# respostas_df.to_excel("docs/respostas_classificacao_sentencas.xlsx", index=False)
respostas_df = pd.read_excel("docs/respostas_classificacao_sentencas.xlsx")

In [3]:
sentencas_danos_ambientais = respostas_df.loc[respostas_df["isDanoAmbiental"] == True]

In [4]:
lista_processos = list(sentencas_danos_ambientais['processoID'].unique())

# Dividindo a lista de processos em 4 partes menores iguais
def divide_lista_em_partes(lista, num_partes):
    if len(lista) % num_partes == 0:
        # Dvisão exata
        tamanho_parte = len(lista) // num_partes
        partes = [lista[i:i + tamanho_parte] for i in range(0, len(lista), tamanho_parte)]
    else:
        # Coloca o resto na última parte
        tamanho_parte = len(lista) // num_partes
        partes = []
        for i in range(0, num_partes):
            if i == num_partes - 1:
                partes.append(lista[i * tamanho_parte:])
            else:
                partes.append(lista[i * tamanho_parte:(i + 1) * tamanho_parte])
    return partes

partes = divide_lista_em_partes(lista_processos, 4)
print("Distribuição de processos entre as partes:")
for i, parte in enumerate(partes):
    print(f"Parte {i + 1}: {len(parte)} processos")
print("Total de processos:", len(lista_processos))

Distribuição de processos entre as partes:
Parte 1: 21 processos
Parte 2: 21 processos
Parte 3: 21 processos
Parte 4: 22 processos
Total de processos: 85


In [5]:
def analisa_sentenca(texto_extraido):
    """
    Função que extrai informações de um texto judicial relacionado a danos ambientais.
    O texto deve ser um texto bruto que descreve um processo judicial relacionado a algum dano ambiental.
    """
    
    prompt = f"""
        SYSTEM: Você é meu assistente especialista em análise e extração de elementos de textos judiciais. Você irá realizar extrações especificamente sobre
        danos socioambientais de diversos tipos, pensando em futuramente usar a tabela gerada para fazer uma modelagem preditiva de multas para danos socioambientais.

        INSTRUCTIONS:
        Você receberá um texto bruto que descreve um processo judicial relacionado a algum dano ambiental.
        Seu objetivo é analisar esse texto e extrair 15 informações específicas para montar um banco de dados, conforme as instruções abaixo.
        Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão
        de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
        Retorno esperado:

        [0] Número do processo judicial no formato "0000000-00.0000.0.00.0000"
         Se não houver, retorne: NULL
        
        [1] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
         Se não houver, retorne: NULL

        [2] Sigla da Unidade Federativa (UF) (ex: "SP", "MG" etc.)
         Se não houver, retorne: NULL

        [3] Município ou cidade do local afetado. (ex: "São Paulo", "Minas Gerais" etc)
         Se não houver, retorne: NULL

        [4] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
         Se não houver, retorne: NULL

        [5] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
         Se não houver, retorne: NULL

        [6] Tipo de impacto: Categoria do dano (ex.: "Desmatamento de APP", "Derramamento de Petróleo", "Poluição Hídrica").
        Se não houver, retorne: NULL

        [7] Descrição do impacto: Resuma o impacto em até 30 palavras.
        Se não houver, retorne: NULL

        [8] Data do impacto ambiental no formato: DD/MM/AA
         Se não houver, retorne: NULL

        [9] Extensão da área afetada (ex: "15000".)
         Se não houver, retorne: NULL
        Sempre em metros quadrados (m²) ou hectares (ha), então converta se necessário. Não use separadores para milhares

        [10] Unidade de medida da área (ex: "ha" ou "m2")
         Se não houver, retorne: NULL

        [11] Houve compensação não monetária atribuída (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
         Se houver, retorne: "True"
         Se não houver, retorne: "False"

        [12] Categoria da compensação: "Multas Administrativas", "Compensações Financeiras", "Obrigações de Fazer (com custo)", "Custas Judiciais e Acordos" ou "Valoração Econômica".
         Se não houver, retorne: NULL
         Tente não fugir dessas categorias.

        [13] Tipo de multa: (0, 1 ou 2)
        Se for uma única multa aplicada sobre o responsável, coloque 0
        Se for algo como uma multa diária, coloque 1.
        Se forem os dois, como uma multa imediata e um pagamento diário, coloque 2

        [14] Valor completo da multa/ressarcimento/compensação ou qualquer outro termo similar de condenação monetária pelo dano ambiental para o Tipo 0 ou 2: (ex: "123000.00", "999999.99")
        Caso o item 12 seja 0, complete essa coluna com o valor total da multa aplicada. Inclua eventuais danos morais difusos, coletivos e materiais se houver.
        Caso o item 12 seja 1, deixe como NULL
        Caso o item 12 seja 2, complete essa coluna com o valor total da multa imediata aplicada.

        [15] Valor da multa/ressarcimento/compensação ou qualquer outro termo similar de condenação monetária pelo dano ambiental diária para o Tipo 1 ou 2: (ex: "123000.00", "10000.50")
        Caso o item 12 seja 1 ou 2, complete essa coluna com o valor diário a ser pago pelo responsável. Inclua eventuais danos morais difusos, coletivos e materiais se houver.
        Caso o item 12 seja 0, deixe como NULL

        IMPORTANTE: A string será usada para um dataframe posteriormente. Logo:
        NÃO inclua explicações, formatações extras, aspas ou colchetes na resposta.
        NÃO deixe nenhuma casa em branco ou vazia. SEMPRE coloque NULL caso não encontre no texto.
        NÃO inclua o texto original, nem explicações, apenas a string limpa.
        A ordem dos elementos PRECISA SER RESPEITADA!! NENHUM ELEMENTO PODE VIR ANTES DA SUA POSIÇÃO!!

        USER:
        Texto para análise:
        {texto_extraido}
    """
    
    class FormatoResposta2(BaseModel):
        numero_processo: str
        georreferencia: str
        uf: str
        municipio: str
        responsavel: str
        categoria_responsavel: str
        tipo_impacto: str
        descricao_impacto: str
        data_impacto: str
        area_afetada: str
        unidade_area: str
        houve_compensacao: bool
        categoria_compensacao: str
        tipo_multa: int | str
        valor_multa: float | str
        valor_multa_diaria: float | str

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': FormatoResposta2,
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    time.sleep(2)
    return resposta

In [6]:
# Analisando os processos com dano ambiental
respostas_danos_ambientais = []
for id in tqdm(lista_processos):
    linha = sentencas_danos_ambientais.loc[sentencas_danos_ambientais['processoID'] == id]
    texto = ''
    linha = linha.iloc[0] # extrai a linha como Series

    processoAnexoID = linha['processoAnexoID']
    link = linha['link_referencia']
    response = requests.get(link)
    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    
    if 'pdf' in content_type:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto += page.get_text()
    
    elif 'html' in content_type:
        soup = BeautifulSoup(response.content, 'html.parser')
        texto += soup.get_text()

    resposta_prompt = analisa_sentenca(texto)
    resposta_prompt = json.loads(resposta_prompt.text)
    resposta_prompt["processoAnexoID"] = processoAnexoID
    resposta_prompt["link_referencia"] = link
    resposta_prompt["processoID"] = id
    respostas_danos_ambientais.append(resposta_prompt)

respostas_danos_ambientais

 31%|███       | 26/85 [02:48<06:21,  6.47s/it]


KeyboardInterrupt: 

In [7]:
# Transformando a lista de respostas de danos ambientais da primeira parte em um DataFrame
respostas_danos_ambientais_df_completo = pd.DataFrame(respostas_danos_ambientais)

# Reorganizando as colunas
respostas_danos_ambientais_df_completo = respostas_danos_ambientais_df_completo[["numero_processo", "processoID", "processoAnexoID", "georreferencia", "uf", "municipio", "responsavel", "categoria_responsavel", "tipo_impacto", "descricao_impacto", "data_impacto", "area_afetada", "unidade_area", "houve_compensacao", "categoria_compensacao", "tipo_multa", "valor_multa", "valor_multa_diaria", "link_referencia"]]

respostas_danos_ambientais_df_completo.head()

,numero_processo,processoID,processoAnexoID,georreferencia,uf,municipio,responsavel,categoria_responsavel,tipo_impacto,descricao_impacto,data_impacto,area_afetada,unidade_area,houve_compensacao,categoria_compensacao,tipo_multa,valor_multa,valor_multa_diaria,link_referencia
0,0600357-89.2021.8.04.5600,474474022,2344968081,NULL,AM,Manicoré,DESTAQUE INDUSTRIA E COMERCIO DE MADEIRAS LTDA,Pessoa Jurídica,Desmatamento,"Depósito irregular de 288,03 metros cúbicos de...",30/04/2016,NULL,NULL,True,Obrigações de Fazer (com custo),0,NULL,NULL,http://jud-anexos.digesto.com.br/0cacd6d80c499...
1,0600384-72.2021.8.04.5600,474474806,2294638461,07°43’11.00” S e 61°28’11.00” O,AM,Manicoré,CARLOS ALEXANDRE ROSSI,Pessoa Física,Desmatamento de APP,"Destruição de 121,15 hectares de floresta nati...",16/06/2016,121.15,ha,False,Obrigações de Fazer (com custo),0,NULL,NULL,http://jud-anexos.digesto.com.br/90df1f9ac9917...
2,0600592-22.2022.8.04.5600,682231747,1915313002,NULL,AM,Manicoré,C R MADEIREIRA AMAZONAS LTDA-EPP,Pessoa Jurídica,Lançamento e queima de rejeitos industriais,A empresa lançou e queimou rejeitos de indústr...,07/11/2018,NULL,NULL,True,Obrigações de Fazer (com custo),2,24240.00,10000.00,http://jud-anexos.digesto.com.br/209b98634c9c4...
3,1000009-75.2018.4.01.3903,311524686,269166848,NULL,PA,Altamira,MARCIO SILVA VALE,Pessoa Física,Desmatamento,Desmatamento e impedimento da regeneração da c...,NULL,NULL,NULL,False,NULL,0,NULL,NULL,http://jud-anexos.digesto.com.br/7cef4bd654c35...
4,1000305-06.2018.4.01.3901,326977847,1526470307,NULL,PA,Marabá,VALE S.A.,Pessoa Jurídica,Desmatamento de APP,Supressão de castanhais e poluição hídrica afe...,NULL,NULL,NULL,True,Custas Judiciais e Acordos,0,1364000.00,NULL,http://jud-anexos.digesto.com.br/d5d876cd4f1b1...


In [ ]:
# Exportando df de classificação de sentenças para um arquivo Excel
# respostas_danos_ambientais_df_parte0.to_excel("docs/respostas_danos_ambientais_df_parte0.xlsx", index=False)

In [8]:
respostas_danos_ambientais_df_completo.to_excel("docs/respostas_danos_ambientais_df_completo.xlsx", index=False)

# Extracao Nasser

In [13]:
# --- CONFIGURAÇÕES DO BANCO DE DADOS SQLITE ---
DB_FILE_NAME = "meu_banco_gemini.db"
TABLE_PROCESSOS = "processos_analisados_gemini"

# Colunas na ordem exata que você especificou para o DataFrame final
# Estas são as colunas que você espera que a API Gemini retorne (ou que você preencha)
# mais as colunas 'processoID', 'processoAnexoID', 'link_referencia'
COLUNAS_DATAFRAME_FINAL = [
    "numero_processo", "processoID", "processoAnexoID", "georreferencia", "uf", "municipio",
    "responsavel", "categoria_responsavel", "tipo_impacto", "descricao_impacto",
    "data_impacto", "area_afetada", "unidade_area", "houve_compensacao",
    "categoria_compensacao", "tipo_multa", "valor_multa", "valor_multa_diaria",
    "link_referencia"
]

In [14]:
# --- FUNÇÕES AUXILIARES SQLITE ---

def criar_tabela_sqlite():
    conn = None
    try:
        conn = sqlite3.connect(DB_FILE_NAME)
        cursor = conn.cursor()
        # Criar colunas com tipo TEXT para flexibilidade. processoID será a chave primária.
        colunas_sql = ", ".join([f'"{col}" TEXT' for col in COLUNAS_DATAFRAME_FINAL if col != "processoID"])
        # processoID é especial e usado como PRIMARY KEY
        cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE_PROCESSOS} (
            "processoID" TEXT PRIMARY KEY,
            {colunas_sql}
        )
        """)
        conn.commit()
    except sqlite3.Error as e:
        print(f"Erro ao criar/conectar tabela SQLite: {e}")
    finally:
        if conn:
            conn.close()

def get_ids_processados_sqlite():
    conn = None
    ids_processados = set()
    # Verificar se o arquivo do banco de dados existe antes de tentar conectar
    if not os.path.exists(DB_FILE_NAME):
        criar_tabela_sqlite() # Cria a tabela se o DB não existir
        return ids_processados

    try:
        conn = sqlite3.connect(DB_FILE_NAME)
        cursor = conn.cursor()
        # Verifica se a tabela existe antes de tentar selecionar
        cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{TABLE_PROCESSOS}';")
        if cursor.fetchone() is None:
            # Tabela não existe, então crie-a
            conn.close() # Fecha a conexão atual para evitar problemas
            criar_tabela_sqlite()
            return ids_processados # Retorna conjunto vazio pois nada foi processado

        cursor.execute(f"SELECT processoID FROM {TABLE_PROCESSOS}")
        rows = cursor.fetchall()
        # Os IDs no seu DataFrame e lista_processos podem ser int ou str.
        # Para consistência, vamos tratar como string ao comparar.
        ids_processados = {str(row[0]) for row in rows}
    except sqlite3.Error as e:
        print(f"Erro ao ler IDs do SQLite: {e}")
    finally:
        if conn:
            conn.close()
    return ids_processados

def inserir_dados_processo_sqlite(dados_dict):
    conn = None
    try:
        conn = sqlite3.connect(DB_FILE_NAME)
        cursor = conn.cursor()
        
        # Garantir que todos os campos de COLUNAS_DATAFRAME_FINAL existam no dict, preenchendo com None se faltar
        # e convertendo todos os valores para string para inserção segura como TEXT
        valores_ordenados = []
        for col_nome in COLUNAS_DATAFRAME_FINAL:
            valor = dados_dict.get(col_nome)
            valores_ordenados.append(str(valor) if valor is not None else None)

        cols_string = ", ".join([f'"{col}"' for col in COLUNAS_DATAFRAME_FINAL])
        placeholders = ", ".join(["?"] * len(COLUNAS_DATAFRAME_FINAL))
        
        # Usar INSERT OR REPLACE para atualizar caso já exista (embora a lógica de pular já deva cobrir)
        # ou INSERT OR IGNORE para simplesmente pular se já existir.
        # Como estamos checando `ids_processados` antes, um INSERT simples seria suficiente,
        # mas OR IGNORE é mais seguro contra race conditions ou lógicas imperfeitas.
        cursor.execute(f"INSERT OR REPLACE INTO {TABLE_PROCESSOS} ({cols_string}) VALUES ({placeholders})",
                       valores_ordenados)
        conn.commit()
    except sqlite3.Error as e:
        print(f"Erro ao inserir dados no SQLite para processoID {dados_dict.get('processoID')}: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

def limpar_resposta_json_gemini(texto_json_bruto: str) -> str:
    """Remove marcadores ```json ... ``` de respostas da API Gemini."""
    limpo = texto_json_bruto.strip()
    if limpo.startswith("```json"):
        limpo = limpo[7:]
    elif limpo.startswith("```"):
        limpo = limpo[3:]
    if limpo.endswith("```"):
        limpo = limpo[:-3]
    return limpo.strip()

In [15]:
# Garante que a tabela exista antes de começar
criar_tabela_sqlite()

In [16]:
ids_ja_processados_no_db = get_ids_processados_sqlite()
print(f"Total de IDs na lista original: {len(lista_processos)}")
print(f"IDs já processados e salvos no banco de dados: {len(ids_ja_processados_no_db)}")

# Filtra a lista_processos para rodar apenas os que ainda não estão no DB
# Convertendo os IDs da lista_processos para string para comparação consistente
lista_processos_pendentes = [str(pid) for pid in lista_processos if str(pid) not in ids_ja_processados_no_db]

print(f"IDs pendentes para processamento: {len(lista_processos_pendentes)}")

respostas_danos_ambientais = [] # Sua lista original para acumular resultados da rodada atual


Total de IDs na lista original: 85
IDs já processados e salvos no banco de dados: 0
IDs pendentes para processamento: 85


In [17]:
if not lista_processos_pendentes:
    print("Todos os processos já foram analisados e estão no banco de dados.")
else:
    for id_str in tqdm(lista_processos_pendentes, desc="Analisando processos pendentes"):
        # O 'id' original pode ser int ou str. Vamos garantir que estamos pegando do DataFrame corretamente.
        # Tentamos como string primeiro, depois como int se o DataFrame usar int.
        try:
            linha = sentencas_danos_ambientais.loc[sentencas_danos_ambientais['processoID'].astype(str) == id_str]
        except KeyError: # Se a coluna 'processoID' não for string e falhar a conversão/comparação
             # Tenta converter o id_str para o tipo da coluna 'processoID' no DataFrame
            id_original_tipo = type(sentencas_danos_ambientais['processoID'].iloc[0])
            try:
                id_convertido = id_original_tipo(id_str)
                linha = sentencas_danos_ambientais.loc[sentencas_danos_ambientais['processoID'] == id_convertido]
            except Exception as e_conv:
                print(f"  AVISO: Não foi possível encontrar o processoID '{id_str}' (ou convertido) no DataFrame 'sentencas_danos_ambientais'. Erro de conversão: {e_conv}. Pulando.")
                continue

        if linha.empty:
            print(f"  AVISO: ProcessoID '{id_str}' não encontrado em 'sentencas_danos_ambientais'. Pulando.")
            continue

        texto_extraido = ''
        linha_serie = linha.iloc[0] # extrai a linha como Series

        processoAnexoID = linha_serie['processoAnexoID']
        link = linha_serie['link_referencia']
        
        print(f"\nProcessando ID: {id_str} | AnexoID: {processoAnexoID} | Link: {link}")

        try:
            response = requests.get(link, timeout=60) # Timeout maior para downloads
            response.raise_for_status() # Levanta exceção para erros HTTP 4xx/5xx
            response.encoding = response.apparent_encoding if response.apparent_encoding else 'utf-8' # Melhor detecção de encoding
            content_type = response.headers.get('Content-Type', '').lower()
            
            if 'pdf' in content_type:
                # print("  Extraindo texto de PDF...")
                with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                    tmp_file.write(response.content)
                    tmp_path = tmp_file.name
                try:
                    with pymupdf.open(tmp_path) as doc:
                        for page_num, page in enumerate(doc):
                            texto_extraido += page.get_text("text") # "text" para melhor extração
                except Exception as e_pdf:
                    print(f"  ERRO ao processar PDF de {link}: {e_pdf}. Pulando este processo.")
                    if os.path.exists(tmp_path): os.remove(tmp_path)
                    continue # Pula para o próximo ID
                finally:
                    if os.path.exists(tmp_path): os.remove(tmp_path)
            
            elif 'html' in content_type or 'text/plain' in content_type or not content_type : # Tenta HTML ou texto puro
                # print("  Extraindo texto de HTML/TEXT...")
                soup = BeautifulSoup(response.content, 'html.parser')
                # Remove tags de script e style
                for script_or_style in soup(["script", "style"]):
                    script_or_style.decompose()
                texto_extraido += soup.get_text(separator='\n', strip=True)
            
            else: # Se não for PDF nem HTML/TEXT conhecido, tenta ler como texto simples
                print(f"  AVISO: Content-Type '{content_type}' não é PDF nem HTML. Tentando ler como texto direto.")
                texto_extraido += response.text

            if not texto_extraido.strip():
                print(f"  AVISO: Nenhum texto foi extraído de {link}. Pulando análise Gemini.")
                continue

            # print(f"  Texto extraído (primeiros 100 chars): {texto_extraido[:100].replace(chr(10), ' ')}...")
            
            # Chamada à sua função que interage com a API Gemini
            # Esta função deve retornar um objeto com um atributo .text contendo o JSON
            resposta_api_obj = analisa_sentenca(texto_extraido) # SUA FUNÇÃO REAL AQUI

            # Limpa e parseia o JSON da resposta
            json_string_limpo = limpar_resposta_json_gemini(resposta_api_obj.text)
            dados_da_api = json.loads(json_string_limpo)
            
            # Monta o dicionário completo para este processo
            # Garante que o 'processoID' seja o mesmo usado para busca (string)
            dados_completos_processo = {"processoID": str(id_str)} 
            dados_completos_processo.update(dados_da_api) # Adiciona dados da API
            
            # Adiciona/sobrescreve as informações que você gerencia manualmente
            dados_completos_processo["processoAnexoID"] = processoAnexoID
            dados_completos_processo["link_referencia"] = link
            
            # Adiciona à lista da rodada atual (como no seu código original)
            respostas_danos_ambientais.append(dados_completos_processo)
            
            # Salva no banco de dados SQLite IMEDIATAMENTE
            inserir_dados_processo_sqlite(dados_completos_processo)
            print(f"  Processo {id_str} analisado e salvo no banco de dados.")

        except requests.exceptions.RequestException as e_req:
            print(f"  ERRO DE REQUEST ao acessar {link}: {e_req}. Pulando este processo.")
        except json.JSONDecodeError as e_json:
            print(f"  ERRO AO DECODIFICAR JSON da API para o processo {id_str}. Resposta: {getattr(resposta_api_obj, 'text', 'N/A')[:200]}. Erro: {e_json}. Pulando.")
        except AttributeError as e_attr: # Caso analisa_sentenca não retorne .text
            print(f"  ERRO: A função 'analisa_sentenca' não retornou um objeto com atributo '.text' para o processo {id_str}. Erro: {e_attr}. Pulando.")
        except Exception as e_geral:
            print(f"  ERRO INESPERADO ao processar ID {id_str}: {e_geral}. Pulando este processo.")
            # import traceback
            # print(traceback.format_exc()) # Para debug mais detalhado se necessário
    
    print("\nProcessamento dos IDs pendentes concluído.")

Analisando processos pendentes:   0%|          | 0/85 [00:00<?, ?it/s]


Processando ID: 474474022 | AnexoID: 2344968081 | Link: http://jud-anexos.digesto.com.br/0cacd6d80c499ae25dcb85380a07c3dd.pdf


Analisando processos pendentes:   1%|          | 1/85 [00:06<08:57,  6.40s/it]

  Processo 474474022 analisado e salvo no banco de dados.

Processando ID: 474474806 | AnexoID: 2294638461 | Link: http://jud-anexos.digesto.com.br/90df1f9ac9917f6df4b9f91915b3a8bd.pdf


Analisando processos pendentes:   2%|▏         | 2/85 [00:11<07:55,  5.72s/it]

...

  Processo 494408691 analisado e salvo no banco de dados.

Processando ID: 494410810 | AnexoID: 269258456 | Link: http://jud-anexos.digesto.com.br/e55b90bdcd194e97c68f4c8754c17646.pdf


Analisando processos pendentes:  99%|█████████▉| 84/85 [09:19<00:06,  6.19s/it]

  Processo 494410810 analisado e salvo no banco de dados.

Processando ID: 549929890 | AnexoID: 1787605420 | Link: http://jud-anexos.digesto.com.br/dce333e4d851f163a94f62103aef5f44.pdf


Analisando processos pendentes: 100%|██████████| 85/85 [09:25<00:00,  6.66s/it]

  Processo 549929890 analisado e salvo no banco de dados.

Processamento dos IDs pendentes concluído.


In [18]:
print("\nCarregando todos os dados do banco de dados para o DataFrame final...")
conn = None
try:
    conn = sqlite3.connect(DB_FILE_NAME)
    # Lê todas as colunas na ordem definida em COLUNAS_DATAFRAME_FINAL
    # Assegura que todas as colunas sejam lidas, mesmo que algumas estejam vazias para alguns registros
    col_select_str = ", ".join([f'"{col}"' for col in COLUNAS_DATAFRAME_FINAL])
    
    # Verifica se a tabela existe antes de tentar ler
    cursor_check = conn.cursor()
    cursor_check.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{TABLE_PROCESSOS}';")
    if cursor_check.fetchone() is None:
        print(f"A tabela {TABLE_PROCESSOS} não existe no banco de dados. Nenhum dado para carregar.")
        respostas_danos_ambientais_df_completo = pd.DataFrame(columns=COLUNAS_DATAFRAME_FINAL)
    else:
        respostas_danos_ambientais_df_completo = pd.read_sql_query(f"SELECT {col_select_str} FROM {TABLE_PROCESSOS}", conn)
except sqlite3.Error as e:
    print(f"Erro ao ler dados do SQLite para o DataFrame: {e}")
    respostas_danos_ambientais_df_completo = pd.DataFrame(columns=COLUNAS_DATAFRAME_FINAL) # Cria DF vazio em caso de erro
finally:
    if conn:
        conn.close()

if not respostas_danos_ambientais_df_completo.empty:
    # A ordem das colunas já deve estar correta devido ao SELECT explícito,
    # mas podemos reconfirmar para garantir, caso a leitura do SQL não preserve 100% a ordem.
    respostas_danos_ambientais_df_completo = respostas_danos_ambientais_df_completo[COLUNAS_DATAFRAME_FINAL]
    
    print(f"\nDataFrame final construído a partir do SQLite com {len(respostas_danos_ambientais_df_completo)} registros.")
    print(respostas_danos_ambientais_df_completo.head())

    output_excel_file = "docs/respostas_danos_ambientais_df_completo.xlsx"
    os.makedirs("docs", exist_ok=True) # Cria o diretório 'docs' se não existir
    respostas_danos_ambientais_df_completo.to_excel(output_excel_file, index=False)
    print(f"\nDataFrame final salvo em: {output_excel_file}")
else:
    print("\nNenhum dado foi carregado do banco de dados para gerar o arquivo Excel.")


Carregando todos os dados do banco de dados para o DataFrame final...

DataFrame final construído a partir do SQLite com 85 registros.
             numero_processo processoID processoAnexoID  \
0  0600357-89.2021.8.04.5600  474474022      2344968081   
1  0600384-72.2021.8.04.5600  474474806      2294638461   
2  0600592-22.2022.8.04.5600  682231747      1915313002   
3  1000009-75.2018.4.01.3903  311524686       269166848   
4  1000305-06.2018.4.01.3901  326977847      1526470307   

                    georreferencia  uf municipio  \
0                             NULL  AM  Manicoré   
1  07°43’11.00” S e 61°28’11.00” O  AM  Manicoré   
2                             NULL  AM  Manicoré   
3                             NULL  PA  Altamira   
4                             NULL  PA    Marabá   

                                      responsavel categoria_responsavel  \
0  DESTAQUE INDUSTRIA E COMERCIO DE MADEIRAS LTDA       Pessoa Jurídica   
1                          CARLOS ALEXANDRE RO